In [1]:
import csv
import re
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import nltk
nltk.download('punkt')
import time

import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zdsfn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def load_and_preprocess(filename):
    #./data_toeic/5000+2000.csv
    f = open(filename, 'r',encoding='utf-8')
    csvReader = csv.reader(f)

    raw_data = []
    raw_sentences = []
    sentences = []
    label = []
    for row in csvReader:
        raw_data.append(row)
    f.close()

    del raw_data[0]
    
    print("총 문제 개수: ", len(raw_data))
    print("총 문장 개수: ", len(raw_data)*4)
    for row_index, row in enumerate(raw_data):
        row_sentence = []
        row_label = []
        hype1 = 0
        hype2 = 0
        for item_index, item in enumerate(row):
            # .뒤에 나오는 것 다 없애기 .이 여러 개 있을지 모르니 마지막 .을 이용하기. 두 문장인 경우 .과 ?과 !이 존재
            # 특수문자 앞에 공백으로 하기
            # 공백 없애기
            # 공백 두개, 세개 -> 한개로 바꾸기
            # _____,----- 연달아 있을시 index 찾아 양쪽 공백 만들기
            # 소문자로 바꾸기

            if item_index == 0:
                #
                index_list = []
                index_of_dot = item.rfind('.')
                index_of_question = item.rfind('?')
                index_of_surprise = item.rfind('!')
                index_list.append(index_of_dot)
                index_list.append(index_of_question)
                index_list.append(index_of_surprise)
                standard = max(index_list)+1
                if standard >0:
                    item = item[:standard]   

                #
                item = item.strip()
                #
                item = item.replace("  ", " ")
                item = item.replace("  ", " ")

                #
                index_of_hype1 = item.find('__')
                index_of_hype2 = item.find('--')
                index_of_hype3 = item.rfind('__')
                index_of_hype4 = item.rfind('--')
                if index_of_hype1 > index_of_hype2:
                    if index_of_hype3 > index_of_hype4:
                        hype1 = index_of_hype1
                        hype2 = index_of_hype3+1
                elif index_of_hype2 > index_of_hype1:
                    if index_of_hype4 > index_of_hype3:
                        hype1 = index_of_hype2
                        hype2 = index_of_hype4+1

                #----- => hype1 = 0 , hype2 = 4
                if hype1 == 0:
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                else:
                    if item[hype1-1] != ' ':
                        item = item[:hype1] + ' ' + item[hype1:]
                        hype1 = hype1+1
                        hype2 = hype2+1
                    if (hype2+1) < len(item):
                        if item[hype2+1] != ' ':
                            item = item[:hype2+1] + ' ' + item[hype2+1:]
                #
                item = item.lower()
                # 문장이 아닐 때
            else:
                 item = item.strip()
            raw_data[row_index][item_index] = item
        #sentence 4개 만들기
        #혹시몰라 raw 저장
        raw_sentences.append(row[0])
        row[0] = row[0].replace(row[0][hype1:hype2+1],"")
        for i in range(1,5):
            sentence = row[0][:hype1] + row[i] + row[0][hype1:]
            row_sentence.append(sentence)

        sentences.append(row_sentence)

        #label 만들기
        count = 0
        for i in range(1,5):
            if row[5] == row[i]:
                row_label.append(1)
                count+=1
            else:
                row_label.append(0)

        #모두 0일 경우 찾아서 1 넣어 주기
        if count!=1:
            if 'A' in row[5]:
                row_label[0]=1
            elif 'B' in row[5]:
                row_label[1]=1
            elif 'C' in row[5]:
                row_label[2]=1
            elif 'D' in row[5]:
                row_label[3]=1

        #레이블에 추가해주기
        label.append(row_label)

    sentences= np.array(sentences)
    sentences= sentences.flatten()
    label = np.array(label)
    label = label.flatten()
    
    return sentences, label

In [18]:
def load_example(filename):
    df = pd.read_csv(filename, error_bad_lines=False)
    sentences = df['SentimentText']
    label = df['Sentiment']
    R_S = []
    R_L = []

    for s in sentences:
        R_S.append(s)
    for l in label:
        R_L.append(l)
        
    print(R_S[:5])
    print(R_L[:5])
    return R_S,R_L

In [3]:
#가공만 된 문장 indexing, embedding 전 입니다.
class ToeicDataset(Dataset):
    def __init__(self, sentences, label):
        self.sentences = sentences
        self.label = label
        self.len = len(self.label)
        
        self.label_list = list(sorted(set(self.label)))
        
    def __getitem__(self, index):
        return self.sentences[index], self.label[index]
    def __len__(self):
        return self.len
    
    def get_labels(self):
        return self.label_list
    
    def get_label(self,id):
        return self.label_list[id]
    def get_label_id(self,label):
        return self.label_list.index(label)

In [4]:
def make_vocab(sentences):
    vocab = set()
    for index, sentence in enumerate(sentences):
        temp = nltk.word_tokenize(sentence)
        vocab.update(temp)
    word_to_ix = {word:i+1 for i, word in enumerate(vocab)}
    word_to_ix['_PAD'] = 0
    word_to_ix['_UNK'] = 1
    vocabsize = len(word_to_ix)
    return vocab, vocabsize, word_to_ix

In [5]:
#make torch
def make_variables(sentences, label):
    
    final_sentences = []
    
    #tokenizing
    for index, sentence in enumerate(sentences):
        final_sentences.append(nltk.word_tokenize(sentence))
    
    #indexing
    for index, sentence in enumerate(final_sentences):
        final_sentences[index]=[word_to_ix[word] for word in sentence]

    #각자의 seq_length 구하기 (미니배치별로 진행)
    seq_lengths = []
    for sentence in final_sentences:
        seq_lengths.append(len(sentence))
    seq_lengths = torch.LongTensor(seq_lengths)
    
    return padding_tensor_sorting(final_sentences,seq_lengths,label)

In [6]:
def padding_tensor_sorting(sentences, seq_lengths, label):
    seq_tensor = torch.zeros((len(sentences), seq_lengths.max())).long()
    print('seq_tensor(max):' , seq_lengths.max())
    for idx, (seq, seq_len) in enumerate(zip(sentences, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    
    if len(seq_tensor) !=1:
        seq_lengths, perm_idx = seq_lengths.sort(0,descending=True)
        seq_tensor = seq_tensor[perm_idx]
    
    target = torch.tensor(label, dtype=torch.long)
    if len(seq_tensor) !=1:
        target = target[perm_idx]
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)

In [7]:
def create_variable(tensor):
    #tensor를 gpu 이용 가능한지
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

In [8]:
def train():
    total_loss = 0
    for i, (sentences, label) in enumerate(train_loader,1):
        input, seq_lengths, target = make_variables(sentences, label)
        print(len(input))
        output = classifier(input, seq_lengths)
        
        loss = criterion(output, target)
        total_loss +=loss.data[0]
        
        classifier.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                epoch,  i *
                len(sentences), len(train_loader.dataset),
                100. * i * len(sentences) / len(train_loader.dataset),
                total_loss / i * len(sentences)))
        
    return total_loss

In [9]:
def test(sentences):
    
    print('evaluating trained model...')
    
    input,seq_lengths, target = make_variables([sentences],[])
    output = classifier(input, seq_lengths)
    print(output)
    pred = output.data.max(1, keepdim = True)[1]
    print(pred)
    label_id = pred.cpu().numpy()[0][0]
    print(label_id)
    print(sentences, " is ", train_dataset.get_label(label_id))

In [10]:
class myModel(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, bidirectional=True ,dropout_p=0.2):
        super(myModel, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.n_vocab = n_vocab
        self.embed_dim = embed_dim
        self.n_classes = n_classes
        self.n_directions = int(bidirectional) + 1
        
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(self.embed_dim, self.hidden_dim,
                            num_layers=self.n_layers,
                            dropout=dropout_p,
                            batch_first=True)
        self.relu = nn.ReLU()
#         self.out = nn.Linear(self.hidden_dim, self.n_classes)
        self.fc1 = nn.Linear(self.hidden_dim, 50)
        self.fc2 = nn.Linear(50, self.n_classes)
    def forward(self, x, seq_lengths):
        
        x = x.t()
#         print(x)
        sen_len = x.size(0)
        batch_size = x.size(1)
        
        embedded = self.embed(x)
        
#         print(embedded.size())
        lstm_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy())
        self.hidden = self._init_hidden(batch_size)
#         print(self.hidden.size())
        self.lstm.flatten_parameters()
        
        lstm_out, self.hidden = self.lstm(lstm_input, self.hidden)
        lstm_out, lengths = pad_packed_sequence(lstm_out)
        
        h_t = self.dropout(self.hidden[-1])
        out = self.relu(self.fc1(h_t[-1]))
        out = self.dropout(out)
        logit = self.fc2(out)
#         h_t = self.dropout(self.hidden[-1])
#         logit = self.out(h_t[-1])
        return F.log_softmax(logit, dim=-1)
    
    def _init_hidden(self, batch_size):
        hidden = torch.zeros((self.n_layers, self.n_directions,
                             batch_size, self.hidden_dim))
        return create_variable(hidden)

In [19]:
if __name__ =="__main__":
    #
    BATCH_SIZE = 64
    EPOCHS = 20
    learning_rate = 0.001
    torch.manual_seed(42)
    
    #load data
#     sentences, label = load_and_preprocess('./data_toeic/5000+2000.csv')
    sentences, label = load_example('./Sentiment Analysis Dataset.csv')
    train_dataset = ToeicDataset(sentences, label)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
    # hyper parameter, user data
    n_layer = 2
    n_classes = 2
    hidden_dim = 100
    embed_dim = 64
    bidirectional = True
    dropout_p = 0.2
    vocab,vocabsize,word_to_ix = make_vocab(sentences)
    
    #make model
    classifier = myModel(n_layer, hidden_dim, vocabsize, embed_dim, n_classes, bidirectional, dropout_p)
    
    #classifier and cuda
    #병렬 처리
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)
    if torch.cuda.is_available():
        classifier.cuda()
    
    optimizer = torch.optim.Adam(classifier.parameters(),lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index = 0)
    
    for epoch in range(1, 2):
        train()

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


['                     is so sad for my APL friend.............', '                   I missed the New Moon trailer...', '              omg its already 7:30 :O', "          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...", '         i think mi bf is cheating on me!!!       T_T']
[0, 0, 1, 0, 0]
seq_tensor(max): tensor(49)
64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if __name__ == '__main__':


seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(31)
64
seq_tensor(max): tensor(32)
64
Train Epoch: 1 [640/1578612 (0%)]	Loss: 35.38
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(30)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(32)
64
Train Epoch: 1 [1280/1578612 (0%)]	Loss: 20.69
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(41)
64
Train Ep

seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(44)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(31)
64
Train Epoch: 1 [15360/1578612 (1%)]	Loss: 1.76
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(35)
64
Train Epoch: 1 [16000/1578612 (1%)]	Loss: 1.69
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(61)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(41)
64
Train Epoch: 1 [16640/1578612 (1%)]	Loss: 1.62
seq_tensor(max): tenso

...

64
seq_tensor(max): tensor(41)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
Train Epoch: 1 [1512960/1578612 (96%)]	Loss: 0.02
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(46)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(41)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(31)
64
seq_tensor(max): tensor(49)
64
seq_tensor(max): tensor(35)
64
Train Epoch: 1 [1513600/1578612 (96%)]	Loss: 0.02
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(31)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(31)
64
Train Epoch: 1 [1514240/1578612 (96%)]	Loss: 0.02
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(37)
64
seq_tensor

seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(42)
64
Train Epoch: 1 [1527680/1578612 (97%)]	Loss: 0.02
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
Train Epoch: 1 [1528320/1578612 (97%)]	Loss: 0.02
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(40)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(33)
64
Train Epoch: 1 [1528960/1578612 (97%)]	Loss: 0.02
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(33)
64
seq_tensor(ma

64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(36)
64
Train Epoch: 1 [1542400/1578612 (98%)]	Loss: 0.02
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(44)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(41)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(43)
64
Train Epoch: 1 [1543040/1578612 (98%)]	Loss: 0.02
seq_tensor(max): tensor(30)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(40)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(49)
64
Train Epoch: 1 [1543680/1578612 (98%)]	Loss: 0.02
seq_tensor

seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(41)
64
seq_tensor(max): tensor(33)
64
Train Epoch: 1 [1557120/1578612 (99%)]	Loss: 0.02
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(40)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(34)
64
Train Epoch: 1 [1557760/1578612 (99%)]	Loss: 0.02
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(39)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(33)
64
Train Epoch: 1 [1558400/1578612 (99%)]	Loss: 0.02
seq_tensor(ma

seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(47)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(36)
64
Train Epoch: 1 [1571840/1578612 (100%)]	Loss: 0.02
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(43)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(38)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(35)
64
Train Epoch: 1 [1572480/1578612 (100%)]	Loss: 0.02
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(34)
64
seq_tensor(max): tensor(33)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(32)
64
seq_tensor(max): tensor(45)
64
seq_tensor(max): tensor(36)
64
seq_tensor(max): tensor(35)
64
seq_tensor(max): tensor(37)
64
seq_tensor(max): tensor(33)
64
Train Epoch: 1 [1573120/1578612 (100%)]	Loss: 0.02
seq_tensor

In [22]:
# test("He has received top performance reviews since he joined the sales department two years ago.")
# test("He has received top performance reviews despite he joined the sales department two years ago.")
# test("He has received top performance reviews except he joined the sales department two years ago.")
# test("He has received top performance reviews during he joined the sales department two years ago.")

# test("All clothing sold in my company is made from natural materials and contains no synthetic dyes.")
# test("All clothing sold in my company is made from natural materials and contains no immediate dyes.")
# test("All clothing sold in my company is made from natural materials and contains no reasonable dyes.")
# test("All clothing sold in my company is made from natural materials and contains no assumed dyes.")


test("it's good")

test("it's bad")

test("so bored movie")

evaluating trained model...
seq_tensor(max): tensor(3)
tensor([[-70.2274,   0.0000]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[1]], device='cuda:0')
1
it's good  is  1
evaluating trained model...
seq_tensor(max): tensor(3)
tensor([[-146.1400,    0.0000]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[1]], device='cuda:0')
1
it's bad  is  1
evaluating trained model...
seq_tensor(max): tensor(3)
tensor([[-173.2208,    0.0000]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
tensor([[1]], device='cuda:0')
1
so bored movie  is  1
